<a href="https://colab.research.google.com/github/lulu0o6/Super-Literature-Search/blob/main/7Nov2025_Literature_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================================
# 🔍 Enhanced Literature Collector with 400+ High-Impact Journals
# Keyword: GLP-1 RA depression
# Date: 7Nov2025文献检索
# ===============================================

!pip install requests beautifulsoup4 googletrans==4.0.0-rc1 python-docx tqdm scholarly feedparser

import requests, re, datetime, urllib.parse, time, json, os, pickle
from bs4 import BeautifulSoup
from googletrans import Translator
from docx import Document
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from tqdm import tqdm
import feedparser
from difflib import SequenceMatcher

# ---------- CONFIG ----------
KEYWORDS = ["GLP-1", "CNS"]
LOGIC_OPERATOR = "AND"
START_YEAR = 2020
END_YEAR = 2025
MAX_RESULTS_PER_SOURCE = 30
ENTREZ_EMAIL = "brovo02@gmail.com"

# 日期格式（7Nov2025）
DATE_PREFIX = datetime.datetime.now().strftime("%-d%b%Y")  # Windows系统可能需要修改
FILE_BASENAME = f"{DATE_PREFIX}文献检索"

# 创建数据保存目录
DATA_DIR = "/content/literature_data"
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

# 数据保存文件路径
SEARCH_RESULTS_FILE = f"{DATA_DIR}/search_results.pkl"
FINAL_DATA_FILE = f"{DATA_DIR}/final_data.pkl"
PROGRESS_FILE = f"{DATA_DIR}/progress.json"

# 扩展的高影响因子期刊列表（400+期刊）
HIGH_IMPACT_JOURNALS = [
    # === 顶级综合期刊 (30个) ===
    "Nature", "Science", "Cell", "Lancet", "New England Journal of Medicine",
    "JAMA", "BMJ", "Nature Medicine", "Nature Reviews Drug Discovery",
    "Science Translational Medicine", "Nature Communications", "PNAS",
    "Nature Reviews Immunology", "Nature Reviews Molecular Cell Biology",
    "Cell Research", "Science Advances", "Nature Methods", "Nature Biotechnology",
    "Nature Genetics", "Nature Cell Biology", "Nature Structural & Molecular Biology",
    "Cell Stem Cell", "Cell Metabolism", "Molecular Cell", "Developmental Cell",
    "Current Biology", "eLife", "PLOS Biology", "EMBO Journal", "EMBO Reports",

    # === 精神病学与神经科学 (100个) ===
    "Nature Neuroscience", "Molecular Psychiatry", "American Journal of Psychiatry",
    "Journal of Clinical Psychiatry", "Neuropsychopharmacology", "Biological Psychiatry",
    "JAMA Psychiatry", "World Psychiatry", "Schizophrenia Bulletin",
    "Journal of Neuroscience", "Brain", "Neuron", "Nature Reviews Neuroscience",
    "Lancet Psychiatry", "Lancet Neurology", "JAMA Neurology", "Annals of Neurology",
    "Neurology", "Brain Stimulation", "Translational Psychiatry", "Cerebral Cortex",
    "Journal of Neurochemistry", "Glia", "NeuroImage", "Human Brain Mapping",
    "Biological Psychiatry: Cognitive Neuroscience and Neuroimaging",
    "Neuropsychologia", "Journal of Cognitive Neuroscience", "Cognitive Neuropsychiatry",
    "Depression and Anxiety", "Journal of Affective Disorders", "Bipolar Disorders",
    "Acta Psychiatrica Scandinavica", "Psychological Medicine", "British Journal of Psychiatry",
    "Journal of Psychiatric Research", "Psychiatry Research", "Progress in Neuro-Psychopharmacology",
    "European Neuropsychopharmacology", "International Journal of Neuropsychopharmacology",
    "Psychopharmacology", "Journal of Psychopharmacology", "Neuroscience",
    "Neuroscience and Biobehavioral Reviews", "Frontiers in Neuroscience",
    "Frontiers in Psychiatry", "Frontiers in Neurology", "Journal of Neurology",
    "Multiple Sclerosis Journal", "Parkinsonism & Related Disorders", "Alzheimer's & Dementia",
    "Journal of Alzheimer's Disease", "Neurobiology of Aging", "Neurobiology of Disease",
    "Molecular Neurobiology", "Cellular and Molecular Neurobiology", "Neurotherapeutics",
    "Experimental Neurology", "Journal of Neurotrauma", "Stroke",
    "Annals of Clinical and Translational Neurology", "Journal of the Neurological Sciences",
    "Clinical Neurophysiology", "Epilepsia", "Seizure", "Headache",
    "Journal of Headache and Pain", "Pain", "European Journal of Pain",
    "Journal of Pain", "Neuropsychology", "Journal of the International Neuropsychological Society",
    "Archives of Clinical Neuropsychology", "Clinical Psychology Review",
    "Journal of Abnormal Psychology", "Journal of Consulting and Clinical Psychology",
    "Clinical Psychological Science", "Behaviour Research and Therapy",
    "Journal of Behavior Therapy and Experimental Psychiatry", "Cognitive Therapy and Research",
    "Mindfulness", "Journal of Contextual Behavioral Science", "Psychoneuroendocrinology",
    "Stress", "Journal of Traumatic Stress", "European Archives of Psychiatry and Clinical Neuroscience",
    "Australian and New Zealand Journal of Psychiatry", "Canadian Journal of Psychiatry",
    "General Hospital Psychiatry", "Psychosomatic Medicine", "Journal of Psychosomatic Research",
    "Psychotherapy and Psychosomatics", "Comprehensive Psychiatry", "Addiction",
    "Drug and Alcohol Dependence", "Alcoholism: Clinical and Experimental Research",

    # === 内分泌与代谢 (80个) ===
    "Diabetes Care", "Journal of Clinical Endocrinology & Metabolism",
    "Molecular Metabolism", "Diabetes", "Endocrine Reviews", "Cell Metabolism",
    "Nature Reviews Endocrinology", "Lancet Diabetes & Endocrinology",
    "Diabetologia", "Diabetes Obesity and Metabolism", "Metabolism: Clinical and Experimental",
    "Endocrinology", "Journal of Endocrinology", "European Journal of Endocrinology",
    "Clinical Endocrinology", "Hormone and Metabolic Research", "Thyroid",
    "Journal of Thyroid Research", "Obesity", "International Journal of Obesity",
    "Obesity Reviews", "American Journal of Physiology-Endocrinology and Metabolism",
    "Frontiers in Endocrinology", "Journal of Diabetes Research", "Diabetes Technology & Therapeutics",
    "Pediatric Diabetes", "Diabetes Research and Clinical Practice", "Journal of Diabetes",
    "Journal of Diabetes and its Complications", "Current Diabetes Reports",
    "Diabetes/Metabolism Research and Reviews", "Experimental and Clinical Endocrinology & Diabetes",
    "Endocrine", "Endocrine Practice", "Journal of the Endocrine Society",
    "Reviews in Endocrine and Metabolic Disorders", "Best Practice & Research Clinical Endocrinology & Metabolism",
    "Molecular and Cellular Endocrinology", "Peptides", "Regulatory Peptides",
    "Journal of Peptide Science", "Neuroendocrinology", "Psychoneuroendocrinology",
    "Journal of Neuroendocrinology", "Frontiers in Neuroendocrinology",
    "Growth Hormone & IGF Research", "Bone", "Journal of Bone and Mineral Research",
    "Osteoporosis International", "Calcified Tissue International", "Bone Research",
    "Journal of Clinical Densitometry", "Journal of Steroid Biochemistry and Molecular Biology",
    "Steroids", "Molecular and Cellular Biochemistry", "Biochimica et Biophysica Acta (BBA) - Molecular Basis of Disease",
    "Journal of Biological Chemistry", "Biochemical Journal", "FEBS Journal",
    "Journal of Molecular Endocrinology", "Endocrine-Related Cancer",
    "Journal of Clinical Oncology", "Cancer Research", "Clinical Cancer Research",
    "Molecular Cancer Research", "Cancer Cell", "Nature Reviews Cancer",
    "Journal of the National Cancer Institute", "Cancer Discovery",
    "CA: A Cancer Journal for Clinicians", "Annals of Oncology",
    "European Journal of Cancer", "British Journal of Cancer",
    "International Journal of Cancer", "Carcinogenesis",

    # === 药理学与药物发现 (70个) ===
    "Nature Reviews Drug Discovery", "Pharmacological Reviews",
    "Clinical Pharmacology & Therapeutics", "British Journal of Pharmacology",
    "Journal of Pharmacology and Experimental Therapeutics", "Molecular Pharmacology",
    "Pharmacology & Therapeutics", "Trends in Pharmacological Sciences",
    "Drug Discovery Today", "Expert Opinion on Drug Discovery",
    "Journal of Medicinal Chemistry", "Bioorganic & Medicinal Chemistry",
    "European Journal of Medicinal Chemistry", "Medicinal Research Reviews",
    "Drug Metabolism Reviews", "Xenobiotica", "Clinical Pharmacokinetics",
    "British Journal of Clinical Pharmacology", "Journal of Clinical Pharmacy and Therapeutics",
    "Therapeutic Drug Monitoring", "Pharmacogenomics", "Pharmacogenetics and Genomics",
    "Journal of Pharmacokinetics and Pharmacodynamics", "CPT: Pharmacometrics & Systems Pharmacology",
    "AAPS Journal", "Pharmaceutical Research", "International Journal of Pharmaceutics",
    "Journal of Controlled Release", "Advanced Drug Delivery Reviews",
    "Molecular Pharmaceutics", "European Journal of Pharmaceutics and Biopharmaceutics",
    "Journal of Pharmaceutical Sciences", "Pharmaceutics", "Drug Delivery",
    "Expert Opinion on Drug Delivery", "Journal of Drug Targeting",
    "Current Opinion in Pharmacology", "Biochemical Pharmacology",
    "Cellular and Molecular Life Sciences", "Journal of Cellular Physiology",
    "Journal of Biological Chemistry", "Proceedings of the National Academy of Sciences",
    "ACS Chemical Neuroscience", "Journal of Neurochemistry",
    "Neuropharmacology", "Psychopharmacology", "European Journal of Pharmacology",
    "Naunyn-Schmiedeberg's Archives of Pharmacology", "Pharmacology Biochemistry and Behavior",
    "Journal of Ethnopharmacology", "Planta Medica", "Phytomedicine",
    "Journal of Natural Products", "Marine Drugs", "Frontiers in Pharmacology",
    "Pharmacological Research", "Basic & Clinical Pharmacology & Toxicology",
    "Toxicology and Applied Pharmacology", "Chemical Research in Toxicology",
    "Expert Opinion on Therapeutic Patents", "Drugs", "CNS Drugs",
    "Journal of Antimicrobial Chemotherapy", "Antimicrobial Agents and Chemotherapy",
    "Journal of Infectious Diseases", "Clinical Infectious Diseases",
    "Vaccine", "Expert Review of Vaccines", "mAbs", "Journal of Immunotherapy",

    # === 心血管科学 (60个) ===
    "Journal of the American College of Cardiology", "Circulation",
    "European Heart Journal", "Circulation Research", "Journal of the American Heart Association",
    "Arteriosclerosis, Thrombosis, and Vascular Biology", "Hypertension",
    "Journal of Hypertension", "American Journal of Hypertension",
    "European Journal of Heart Failure", "Journal of Cardiac Failure",
    "Heart", "Journal of the American Society of Echocardiography",
    "European Journal of Echocardiography", "JACC: Cardiovascular Imaging",
    "JACC: Heart Failure", "JACC: Clinical Electrophysiology",
    "Europace", "Heart Rhythm", "Pacing and Clinical Electrophysiology",
    "Cardiovascular Research", "Basic Research in Cardiology",
    "American Journal of Physiology-Heart and Circulatory Physiology",
    "Journal of Molecular and Cellular Cardiology", "Cardiovascular Diabetology",
    "Diabetes and Vascular Disease Research", "Atherosclerosis",
    "Journal of Lipid Research", "Arteriosclerosis",
    "Current Opinion in Lipidology", "Progress in Lipid Research",
    "Clinical Science", "American Journal of Cardiology",
    "International Journal of Cardiology", "Canadian Journal of Cardiology",
    "Journal of Cardiovascular Electrophysiology", "Circulation: Arrhythmia and Electrophysiology",
    "Circulation: Cardiovascular Imaging", "Circulation: Cardiovascular Interventions",
    "Circulation: Heart Failure", "Circulation: Genomic and Precision Medicine",
    "Nature Reviews Cardiology", "Lancet Cardiology",
    "JACC: Basic to Translational Science", "Cardiovascular Drugs and Therapy",
    "Journal of Cardiovascular Pharmacology", "Pharmacological Research",
    "Frontiers in Cardiovascular Medicine", "European Journal of Preventive Cardiology",
    "Journal of Clinical Lipidology", "Nutrition, Metabolism and Cardiovascular Diseases",
    "Obesity Research & Clinical Practice", "Journal of Thoracic and Cardiovascular Surgery",
    "Annals of Thoracic Surgery", "European Journal of Cardio-Thoracic Surgery",
    "Journal of Vascular Surgery", "European Journal of Vascular and Endovascular Surgery",

    # === 分子生物学与细胞生物学 (60个) ===
    "Molecular Cell", "Cell Reports", "Science Advances", "Nature Metabolism",
    "JCI Insight", "Cell Research", "EMBO Journal", "Genes & Development",
    "Developmental Cell", "Current Biology", "eLife", "PLOS Biology",
    "Journal of Cell Biology", "Molecular Biology of the Cell", "Journal of Cell Science",
    "Cell Death & Differentiation", "Autophagy", "Apoptosis", "Journal of Biological Chemistry",
    "Biochemical Journal", "FEBS Letters", "Biochimica et Biophysica Acta (BBA)",
    "Journal of Molecular Biology", "Structure", "Journal of Structural Biology",
    "Current Opinion in Cell Biology", "Trends in Cell Biology", "Nature Cell Biology",
    "Cell Cycle", "Cell Signal", "Cellular Signalling", "Journal of Cellular Biochemistry",
    "Experimental Cell Research", "Cell and Tissue Research", "Histochemistry and Cell Biology",
    "Journal of Histochemistry and Cytochemistry", "Cytokine", "Growth Factors",
    "Stem Cells", "Stem Cell Reports", "Cell Stem Cell", "Nature Stem Cell",
    "Stem Cell Research & Therapy", "Journal of Stem Cells and Regenerative Medicine",
    "Tissue Engineering", "Biomaterials", "Acta Biomaterialia",
    "Journal of Biomedical Materials Research", "Advanced Healthcare Materials",
    "Biofabrication", "Organogenesis", "Developmental Biology",
    "Genesis", "Mechanisms of Development", "Differentiation",
    "Seminars in Cell & Developmental Biology", "International Journal of Developmental Biology",
    "Journal of Developmental Biology", "Frontiers in Cell and Developmental Biology"
]

# 日期格式
translator = Translator()

# ---------- DATA PERSISTENCE FUNCTIONS ----------
def save_progress(stage, data=None):
    """保存进度"""
    progress = {
        'stage': stage,
        'timestamp': datetime.datetime.now().isoformat(),
        'data_size': len(data) if data else 0
    }
    with open(PROGRESS_FILE, 'w') as f:
        json.dump(progress, f)

    if data is not None:
        if stage == 'search_complete':
            with open(SEARCH_RESULTS_FILE, 'wb') as f:
                pickle.dump(data, f)
        elif stage == 'deduplication_complete':
            with open(FINAL_DATA_FILE, 'wb') as f:
                pickle.dump(data, f)

    print(f"💾 Progress saved: {stage}")

def load_progress():
    """加载进度"""
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as f:
            return json.load(f)
    return None

def load_saved_data():
    """加载已保存的数据"""
    data = {}
    if os.path.exists(SEARCH_RESULTS_FILE):
        with open(SEARCH_RESULTS_FILE, 'rb') as f:
            data['search_results'] = pickle.load(f)
    if os.path.exists(FINAL_DATA_FILE):
        with open(FINAL_DATA_FILE, 'rb') as f:
            data['final_data'] = pickle.load(f)
    return data

def check_existing_data():
    """检查是否有已保存的数据"""
    progress = load_progress()
    if progress:
        print(f"📁 Found existing progress: {progress['stage']} at {progress['timestamp']}")

        saved_data = load_saved_data()
        if 'search_results' in saved_data:
            print(f"   Search results: {len(saved_data['search_results'])} items")
        if 'final_data' in saved_data:
            print(f"   Final data: {len(saved_data['final_data'])} items")

        return progress, saved_data
    return None, {}

# ---------- KEYWORD FILTERING FUNCTIONS ----------
def preprocess_text(text):
    """预处理文本用于关键词匹配"""
    if not text:
        return ""
    # 转换为小写，移除标点
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def calculate_keyword_relevance(title, abstract, keywords):
    """计算文献与关键词的相关性分数"""
    if not title or not abstract:
        return 0

    text = preprocess_text(title + " " + abstract)
    if not text:
        return 0

    # 计算关键词出现频率和位置权重
    relevance_score = 0

    for keyword in keywords:
        keyword_lower = keyword.lower()

        # 在标题中出现（最高权重）
        title_lower = preprocess_text(title)
        title_matches = len(re.findall(r'\b' + re.escape(keyword_lower) + r'\b', title_lower))
        relevance_score += title_matches * 3

        # 在摘要中出现
        abstract_lower = preprocess_text(abstract)
        abstract_matches = len(re.findall(r'\b' + re.escape(keyword_lower) + r'\b', abstract_lower))
        relevance_score += abstract_matches * 1

        # 部分匹配（次权重）
        if keyword_lower in text:
            relevance_score += 0.5

    return relevance_score

def filter_by_keywords(items, keywords, min_relevance_score=2):
    """根据关键词筛选文献"""
    print(f"\n🔍 Filtering {len(items)} items by keywords: {keywords}")

    filtered_items = []
    relevance_scores = []

    for item in tqdm(items, desc="Keyword Filtering"):
        title = item.get("Title", "")
        abstract = item.get("Abstract_En", "")

        relevance = calculate_keyword_relevance(title, abstract, keywords)
        relevance_scores.append(relevance)

        if relevance >= min_relevance_score:
            item["Relevance_Score"] = relevance
            filtered_items.append(item)

    # 统计信息
    total_items = len(items)
    filtered_count = len(filtered_items)
    removed_count = total_items - filtered_count

    print(f"📊 KEYWORD FILTERING RESULTS:")
    print(f"  Total items: {total_items}")
    print(f"  After filtering: {filtered_count}")
    print(f"  Removed: {removed_count} ({removed_count/total_items*100:.1f}%)")

    # 显示相关性分数分布
    if relevance_scores:
        avg_score = sum(relevance_scores) / len(relevance_scores)
        max_score = max(relevance_scores)
        min_score = min(relevance_scores)
        print(f"  Relevance scores - Avg: {avg_score:.2f}, Max: {max_score}, Min: {min_score}")

    # 显示被过滤掉的文献示例
    if removed_count > 0:
        print(f"\n📝 Examples of removed items (low relevance):")
        low_relevance_items = [item for item, score in zip(items, relevance_scores) if score < min_relevance_score]
        for i, item in enumerate(low_relevance_items[:3]):
            print(f"  {i+1}. {item['Title'][:80]}... (score: {relevance_scores[items.index(item)]})")

    return filtered_items

def smart_keyword_expansion(keywords):
    """智能扩展关键词（同义词、相关术语）"""
    expanded_keywords = set(keywords)

    # GLP-1 相关术语
    glp1_related = [
        "glp1", "glp-1", "glp 1", "glucagon-like peptide-1", "glucagon like peptide 1",
        "glp-1 receptor", "glp1 receptor", "glp-1 ra", "glp1 ra",
        "glp-1 agonist", "glp1 agonist", "glp-1 analog", "glp1 analog",
        "semaglutide", "liraglutide", "dulaglutide", "exenatide", "lixisenatide"
    ]

    # 抑郁相关术语
    depression_related = [
        "depression", "depressive", "major depressive disorder", "mdd",
        "bipolar", "bipolar disorder", "mood disorder", "affective disorder",
        "mental health", "psychological", "antidepressant", "depression treatment"
    ]

    # 根据输入关键词自动扩展
    for keyword in keywords:
        keyword_lower = keyword.lower()

        if any(term in keyword_lower for term in ['glp', 'peptide']):
            expanded_keywords.update(glp1_related)
        elif any(term in keyword_lower for term in ['depress', 'bipolar', 'mood']):
            expanded_keywords.update(depression_related)

    # 移除原始关键词中可能重复的
    expanded_list = list(expanded_keywords)
    print(f"🔤 Expanded keywords: {len(expanded_list)} terms")
    print(f"   Original: {keywords}")
    print(f"   Expanded: {expanded_list}")

    return expanded_list

def analyze_keyword_distribution(items, keywords):
    """分析关键词在文献中的分布"""
    print(f"\n📈 Analyzing keyword distribution in {len(items)} items...")

    keyword_stats = {}
    for keyword in keywords:
        keyword_lower = keyword.lower()
        count = 0

        for item in items:
            text = preprocess_text(item.get("Title", "") + " " + item.get("Abstract_En", ""))
            if keyword_lower in text:
                count += 1

        keyword_stats[keyword] = count

    # 显示关键词统计
    print("📊 Keyword occurrence statistics:")
    for keyword, count in sorted(keyword_stats.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / len(items)) * 100 if items else 0
        print(f"   {keyword}: {count} items ({percentage:.1f}%)")

    return keyword_stats

# ---------- UTILITIES ----------
def clean_html(text):
    if not text:
        return ""
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def translate_to_cn(text):
    try:
        if not text.strip():
            return ""
        translated = translator.translate(text, src='en', dest='zh-cn').text
        return translated
    except Exception as e:
        print(f"⚠️ Translation failed: {e}")
        return ""

def make_hyperlink(paragraph, url, text):
    """在 Word 段落中插入可点击的超链接"""
    part = paragraph.part
    r_id = part.relate_to(url, "http://schemas.openxmlformats.org/officeDocument/2006/relationships/hyperlink", is_external=True)
    hyperlink = OxmlElement('w:hyperlink')
    hyperlink.set(qn('r:id'), r_id)
    new_run = OxmlElement('w:r')
    rPr = OxmlElement('w:rPr')
    rStyle = OxmlElement('w:rStyle')
    rStyle.set(qn('w:val'), 'Hyperlink')
    rPr.append(rStyle)
    new_run.append(rPr)
    text_elem = OxmlElement('w:t')
    text_elem.text = text
    new_run.append(text_elem)
    hyperlink.append(new_run)
    paragraph._p.append(hyperlink)

def doi_to_link(doi):
    return f"https://doi.org/{doi}" if doi else ""

# ---------- ADVANCED DEDUPLICATION FUNCTIONS ----------
def normalize_text(text):
    """标准化文本用于比较"""
    if not text:
        return ""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def calculate_similarity(title1, title2):
    """计算两个标题的相似度"""
    norm1 = normalize_text(title1)
    norm2 = normalize_text(title2)

    if not norm1 or not norm2:
        return 0

    similarity = SequenceMatcher(None, norm1, norm2).ratio()

    if len(norm1) < 20 or len(norm2) < 20:
        return similarity * 0.8

    return similarity

def is_duplicate(item1, item2, similarity_threshold=0.85):
    """判断两个文献是否重复"""
    if item1["DOI"] and item2["DOI"] and item1["DOI"] == item2["DOI"]:
        return True

    if item1["Link"] and item2["Link"] and item1["Link"] == item2["Link"]:
        return True

    title_similarity = calculate_similarity(item1["Title"], item2["Title"])
    if title_similarity > similarity_threshold:
        return True

    authors1 = normalize_text(item1["Authors"])
    authors2 = normalize_text(item2["Authors"])
    if (authors1 and authors2 and
        len(authors1) > 10 and len(authors2) > 10 and
        authors1 == authors2 and
        item1["Year"] == item2["Year"]):
        return True

    return False

def advanced_deduplication(items):
    """高级去重算法"""
    print(f"\n🧩 Performing advanced deduplication on {len(items)} items...")

    exact_duplicates_removed = 0
    unique_items = []
    seen_keys = set()

    for item in items:
        doi_key = item["DOI"] if item["DOI"] else ""
        title_key = normalize_text(item["Title"])[:100] if item["Title"] else ""
        author_year_key = f"{normalize_text(item['Authors'])[:50]}_{item['Year']}" if item["Authors"] else ""

        if doi_key:
            key = doi_key
        elif title_key and author_year_key:
            key = f"{title_key}_{author_year_key}"
        else:
            key = title_key

        if key and key not in seen_keys:
            seen_keys.add(key)
            unique_items.append(item)
        else:
            exact_duplicates_removed += 1

    print(f"✅ Exact duplicates removed: {exact_duplicates_removed}")
    print(f"📊 After exact deduplication: {len(unique_items)} items")

    similarity_duplicates_removed = 0
    final_items = []
    processed_indices = set()

    for i in tqdm(range(len(unique_items)), desc="Similarity check"):
        if i in processed_indices:
            continue

        current_item = unique_items[i]
        final_items.append(current_item)
        processed_indices.add(i)

        for j in range(i + 1, len(unique_items)):
            if j in processed_indices:
                continue

            if is_duplicate(current_item, unique_items[j]):
                processed_indices.add(j)
                similarity_duplicates_removed += 1
                if (len(unique_items[j]["Abstract_En"]) > len(current_item["Abstract_En"]) and
                    unique_items[j]["Abstract_En"] != "Abstract not available from any source."):
                    final_items[-1] = unique_items[j]

    print(f"✅ Similarity duplicates removed: {similarity_duplicates_removed}")
    print(f"📊 After similarity deduplication: {len(final_items)} items")

    total_removed = exact_duplicates_removed + similarity_duplicates_removed
    print(f"🎯 Total duplicates removed: {total_removed}")

    return final_items

def select_best_version(items):
    """从重复项目中选择最佳版本"""
    if not items:
        return None

    best_item = items[0]

    for item in items[1:]:
        if item["DOI"] and not best_item["DOI"]:
            best_item = item
            continue

        if (len(item["Abstract_En"]) > len(best_item["Abstract_En"]) and
            item["Abstract_En"] != "Abstract not available from any source."):
            best_item = item
            continue

        source_priority = {
            "High-Impact": 5,
            "PubMed": 4,
            "CrossRef": 3,
            "SemanticScholar": 2,
            "Springer": 2
        }

        current_priority = source_priority.get(best_item["Source"], 1)
        new_priority = source_priority.get(item["Source"], 1)

        if new_priority > current_priority:
            best_item = item

    return best_item

def group_and_deduplicate(items):
    """分组并去重"""
    print(f"\n🧩 Grouping and deduplicating {len(items)} items...")

    doi_groups = {}
    no_doi_items = []

    for item in items:
        if item["DOI"]:
            if item["DOI"] not in doi_groups:
                doi_groups[item["DOI"]] = []
            doi_groups[item["DOI"]].append(item)
        else:
            no_doi_items.append(item)

    deduplicated_items = []
    for doi, group in doi_groups.items():
        best_item = select_best_version(group)
        if best_item:
            deduplicated_items.append(best_item)

    print(f"✅ DOI-based groups: {len(doi_groups)}")
    print(f"📊 Items with DOI: {len(deduplicated_items)}")
    print(f"📊 Items without DOI: {len(no_doi_items)}")

    if no_doi_items:
        no_doi_dedup = advanced_deduplication(no_doi_items)
        deduplicated_items.extend(no_doi_dedup)

    return deduplicated_items

# ---------- ENHANCED ABSTRACT COMPLETION FUNCTIONS ----------
def extract_abstract_from_doi_webpage(doi):
    """从DOI原网页抓取摘要"""
    try:
        doi_url = f"https://doi.org/{doi}"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
        }

        response = requests.get(doi_url, headers=headers, timeout=15)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        # 常见摘要选择器（覆盖大多数出版社）
        abstract_selectors = [
            # ScienceDirect
            'div.abstract.author',
            'section.abstract',
            'div.Abstracts',
            # Nature
            'div.c-article-section__content',
            'div#Abs1-content',
            # Wiley
            'section.article-section__abstract',
            'div.abstract-group',
            # Springer
            'div.c-article-section__content',
            'section.Abstract',
            # PubMed Central
            'div.abstract',
            'div.abstract-content',
            # PLOS
            'div.abstract-content',
            # BMC
            'div.Abstract',
            # 通用选择器
            '[class*="abstract"]',
            '[id*="abstract"]',
            'section#abstract',
            'div.abstract-section',
            'p.abstract',
        ]

        # 尝试各种选择器
        for selector in abstract_selectors:
            abstract_elements = soup.select(selector)
            for element in abstract_elements:
                text = element.get_text(strip=True)
                if text and len(text) > 100:  # 确保是真正的摘要
                    # 清理文本
                    text = clean_html(text)
                    if len(text) > 150:
                        print(f"    ✅ Found abstract via selector: {selector}")
                        return text

        # 如果没有通过选择器找到，尝试查找包含"Abstract"的段落
        abstract_keywords = ['abstract', 'summary', '背景', '摘要']
        for keyword in abstract_keywords:
            elements = soup.find_all(['div', 'section', 'p'],
                                   string=re.compile(keyword, re.IGNORECASE))
            for element in elements:
                # 获取相邻的文本内容
                next_sibling = element.find_next_sibling()
                if next_sibling:
                    text = next_sibling.get_text(strip=True)
                    if text and len(text) > 100:
                        text = clean_html(text)
                        if len(text) > 150:
                            print(f"    ✅ Found abstract via keyword: {keyword}")
                            return text

        return ""

    except Exception as e:
        print(f"    ⚠️ DOI webpage extraction failed: {e}")
        return ""

def get_publisher_abstract(doi):
    """通过出版社API获取摘要"""
    try:
        # CrossRef API
        crossref_url = f"https://api.crossref.org/works/{doi}"
        response = requests.get(crossref_url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            abstract = data.get('message', {}).get('abstract')
            if abstract:
                return clean_html(abstract)

        # Semantic Scholar API
        semantic_url = f"https://api.semanticscholar.org/graph/v1/paper/DOI:{doi}?fields=abstract"
        response = requests.get(semantic_url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            abstract = data.get('abstract')
            if abstract:
                return clean_html(abstract)

        return ""
    except:
        return ""

def get_pubmed_abstract(title):
    """通过标题在PubMed中搜索并获取摘要"""
    try:
        clean_title = re.sub(r'[^\w\s]', '', title)
        search_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={urllib.parse.quote(clean_title)}&retmode=json"

        response = requests.get(search_url, timeout=10)
        data = response.json()

        id_list = data.get("esearchresult", {}).get("idlist", [])
        if not id_list:
            return ""

        fetch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={id_list[0]}&retmode=xml"
        fetch_response = requests.get(fetch_url, timeout=10)
        soup = BeautifulSoup(fetch_response.text, "xml")

        abstract_texts = soup.find_all("AbstractText")
        if abstract_texts:
            return clean_html(" ".join([abs_text.text for abs_text in abstract_texts]))

        return ""
    except Exception as e:
        print(f"⚠️ PubMed abstract search failed: {e}")
        return ""

def get_crossref_abstract(title):
    """通过标题在CrossRef中搜索摘要"""
    try:
        q = urllib.parse.quote(title)
        resp = requests.get(f"https://api.crossref.org/works?query.title={q}&rows=3", timeout=15).json()

        if resp.get("message", {}).get("items"):
            for item in resp["message"]["items"]:
                abstract = clean_html(item.get("abstract", ""))
                if abstract and len(abstract) > 100:
                    return abstract
        return ""
    except Exception as e:
        print(f"⚠️ CrossRef abstract search failed: {e}")
        return ""

def get_semantic_scholar_abstract(title):
    """通过标题在Semantic Scholar中搜索摘要"""
    try:
        q = urllib.parse.quote(title)
        url = f"https://api.semanticscholar.org/graph/v1/paper/search?query={q}&limit=3&fields=title,abstract"

        response = requests.get(url, timeout=15)
        data = response.json()

        for paper in data.get("data", []):
            abstract = clean_html(paper.get("abstract", ""))
            if abstract and len(abstract) > 100:
                return abstract
        return ""
    except Exception as e:
        print(f"⚠️ Semantic Scholar abstract search failed: {e}")
        return ""

def get_google_scholar_abstract(title):
    """通过Google Scholar搜索摘要"""
    try:
        q = urllib.parse.quote(title)
        url = f"https://scholar.google.com/scholar?q={q}"

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        response = requests.get(url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.text, 'html.parser')

        abstract_div = soup.find('div', class_='gs_rs')
        if abstract_div:
            abstract = clean_html(abstract_div.text)
            if abstract and len(abstract) > 100:
                return abstract
        return ""
    except Exception as e:
        print(f"⚠️ Google Scholar abstract search failed: {e}")
        return ""

def get_springer_abstract(title):
    """通过SpringerLink搜索摘要"""
    try:
        q = urllib.parse.quote(title)
        url = f"https://api.springernature.com/metadata/json?q=title:{q}&api_key=demo&p=3"

        response = requests.get(url, timeout=15)
        data = response.json()

        for record in data.get("records", []):
            abstract = clean_html(record.get("abstract", ""))
            if abstract and len(abstract) > 100:
                return abstract
        return ""
    except Exception as e:
        print(f"⚠️ Springer abstract search failed: {e}")
        return ""

def force_complete_abstract(item):
    """强制补全摘要，使用多个数据源，包括DOI网页"""
    original_title = item["Title"]
    doi = item.get("DOI", "")

    search_sources = [
        ("PubMed", get_pubmed_abstract),
        ("CrossRef", get_crossref_abstract),
        ("Semantic Scholar", get_semantic_scholar_abstract),
        ("Springer", get_springer_abstract),
        ("Google Scholar", get_google_scholar_abstract),
    ]

    # 如果有DOI，添加DOI相关搜索源
    if doi:
        search_sources.insert(0, ("DOI Publisher", lambda title: get_publisher_abstract(doi)))
        search_sources.insert(1, ("DOI Webpage", lambda title: extract_abstract_from_doi_webpage(doi)))

    best_abstract = ""
    best_source = "Original"
    best_length = 0

    if item["Abstract_En"] and len(item["Abstract_En"]) > 100:
        best_abstract = item["Abstract_En"]
        best_source = item["Source"]
        best_length = len(best_abstract)

    for source_name, search_func in search_sources:
        if best_length > 400:
            break

        print(f"  🔍 Trying {source_name}...")
        try:
            if "DOI" in source_name:
                # DOI相关的搜索不需要标题参数
                abstract = search_func(original_title) if "Publisher" in source_name else search_func(doi)
            else:
                abstract = search_func(original_title)

            if abstract and len(abstract) > best_length + 50:
                best_abstract = abstract
                best_source = source_name
                best_length = len(abstract)
                print(f"    ✅ Found better abstract from {source_name} ({len(abstract)} chars)")

            time.sleep(0.5)

        except Exception as e:
            print(f"    ⚠️ {source_name} search error: {e}")
            continue

    # 如果仍然没有找到合适的摘要
    if not best_abstract or len(best_abstract) < 50:
        if item["Abstract_En"] and len(item["Abstract_En"]) > 10:
            best_abstract = item["Abstract_En"]
            best_source = f"{item['Source']} (Short)"
        else:
            best_abstract = "Abstract not available from any source."
            best_source = "Not Found"

    return best_abstract, best_source

def enhanced_abstract_completion(final_data):
    """增强的摘要补全，包含DOI网页抓取"""
    print("\n🔍 ENHANCED ABSTRACT COMPLETION WITH DOI WEBPAGE EXTRACTION...")
    completion_stats = {"improved": 0, "unchanged": 0, "failed": 0, "doi_extracted": 0}

    for i, d in enumerate(tqdm(final_data, desc="Abstract Completion")):
        if i % 5 == 0:  # 更频繁地显示进度
            print(f"\n[{i+1}/{len(final_data)}] Processing: {d['Title'][:80]}...")
            print(f"  Source: {d['Source']}, DOI: {d.get('DOI', 'No DOI')}")
            print(f"  Original abstract: {len(d['Abstract_En'])} chars")

        original_abstract = d["Abstract_En"]
        new_abstract, new_source = force_complete_abstract(d)

        d["Abstract_En"] = new_abstract
        d["Abstract_Source"] = new_source

        # 统计
        if "DOI" in new_source and len(new_abstract) > len(original_abstract) + 50:
            completion_stats["doi_extracted"] += 1
            completion_stats["improved"] += 1
            print(f"    🎯 DOI webpage extraction successful!")
        elif len(new_abstract) > len(original_abstract) + 100:
            completion_stats["improved"] += 1
        elif new_source == "Not Found":
            completion_stats["failed"] += 1
        else:
            completion_stats["unchanged"] += 1

        # 显示当前结果
        if i % 5 == 0:
            print(f"  Final abstract: {len(new_abstract)} chars from {new_source}")

        # 保存进度
        if i % 10 == 0:
            save_progress('abstract_completion', final_data)

    print(f"\n📊 ENHANCED ABSTRACT COMPLETION STATISTICS:")
    print(f"  Improved: {completion_stats['improved']}")
    print(f"  DOI网页抓取成功: {completion_stats['doi_extracted']}")
    print(f"  Unchanged: {completion_stats['unchanged']}")
    print(f"  Failed: {completion_stats['failed']}")

    return completion_stats

# ---------- SEARCH MODULES ----------
def search_crossref():
    print("\n🔹 Searching CrossRef ...")
    url = "https://api.crossref.org/works"
    params = {"query": " ".join(KEYWORDS), "filter": f"from-pub-date:{START_YEAR},until-pub-date:{END_YEAR}", "rows": MAX_RESULTS_PER_SOURCE}
    try:
        data = requests.get(url, params=params, timeout=30).json()
        items = []
        for i in data.get("message", {}).get("items", []):
            abs_text = clean_html(i.get("abstract", ""))
            item = {
                "Title": i.get("title", [""])[0] if i.get("title") else "No title",
                "Authors": ", ".join([a.get("family", "") for a in i.get("author", [])]) if "author" in i else "",
                "Year": i.get("created", {}).get("date-parts", [[None]])[0][0] if i.get("created") else "",
                "DOI": i.get("DOI", ""),
                "Link": doi_to_link(i.get("DOI", "")),
                "Abstract_En": abs_text,
                "Abstract_Source": "CrossRef" if abs_text else "No Abstract",
                "Source": "CrossRef"
            }
            items.append(item)
        print(f"✅ CrossRef found {len(items)} results")
        return items
    except Exception as e:
        print(f"⚠️ CrossRef search failed: {e}")
        return []

def search_pubmed():
    print("\n🔹 Searching PubMed ...")
    try:
        base = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
        term = "+AND+".join(KEYWORDS)
        esearch = f"{base}esearch.fcgi?db=pubmed&term={term}&retmax={MAX_RESULTS_PER_SOURCE}&mindate={START_YEAR}&maxdate={END_YEAR}&retmode=json"

        response = requests.get(esearch, timeout=30)
        data = response.json()
        ids = data.get("esearchresult", {}).get("idlist", [])

        items = []
        for pmid in tqdm(ids, desc="PubMed Progress"):
            try:
                xml = requests.get(f"{base}efetch.fcgi?db=pubmed&id={pmid}&retmode=xml", timeout=15).text
                soup = BeautifulSoup(xml, "xml")

                title_elem = soup.find("ArticleTitle")
                title = title_elem.text if title_elem else "No title"

                authors = ", ".join([a.text for a in soup.find_all("LastName") if a.text])

                year_tag = soup.find("PubDate")
                year = ""
                if year_tag:
                    year_elem = year_tag.find("Year")
                    year = year_elem.text if year_elem else ""

                abs_text = clean_html(" ".join([a.text for a in soup.find_all("AbstractText")]))

                items.append({
                    "Title": title,
                    "Authors": authors,
                    "Year": year,
                    "DOI": "",
                    "Link": f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/",
                    "Abstract_En": abs_text,
                    "Abstract_Source": "PubMed" if abs_text else "No Abstract",
                    "Source": "PubMed"
                })
            except Exception as e:
                print(f"⚠️ Error processing PubMed ID {pmid}: {e}")
                continue

        print(f"✅ PubMed found {len(items)} results")
        return items
    except Exception as e:
        print(f"⚠️ PubMed search failed: {e}")
        return []

def search_semantic():
    print("\n🔹 Searching Semantic Scholar ...")
    try:
        url = f"https://api.semanticscholar.org/graph/v1/paper/search?query={'+'.join(KEYWORDS)}&year={START_YEAR}-{END_YEAR}&limit={MAX_RESULTS_PER_SOURCE}&fields=title,abstract,authors,year,externalIds,url"
        response = requests.get(url, timeout=30)
        r = response.json()

        items = []
        for p in r.get("data", []):
            doi = p.get("externalIds", {}).get("DOI", "")
            abs_text = clean_html(p.get("abstract", ""))
            authors = ", ".join([a["name"] for a in p.get("authors", [])]) if p.get("authors") else ""

            items.append({
                "Title": p.get("title", "No title"),
                "Authors": authors,
                "Year": p.get("year", ""),
                "DOI": doi,
                "Link": doi_to_link(doi) if doi else p.get("url", ""),
                "Abstract_En": abs_text,
                "Abstract_Source": "SemanticScholar" if abs_text else "No Abstract",
                "Source": "SemanticScholar"
            })
        print(f"✅ Semantic Scholar found {len(items)} results")
        return items
    except Exception as e:
        print(f"⚠️ Semantic Scholar search failed: {e}")
        return []

def search_high_impact_journals():
    """搜索高影响因子期刊 - 每个期刊10篇文献"""
    print("\n🔹 Searching High-Impact Journals (10 articles per journal)...")
    items = []

    # 选项3：按分类搜索（更智能的方式）
    journals_to_search = (
         HIGH_IMPACT_JOURNALS[:30] +  # 顶级综合期刊
         HIGH_IMPACT_JOURNALS[30:130] +  # 精神病学与神经科学
         HIGH_IMPACT_JOURNALS[130:145] +  # 内分泌与代谢
         HIGH_IMPACT_JOURNALS[210:225] +  # 药理学与药物发现
         HIGH_IMPACT_JOURNALS[280:295] +  # 心血管科学
         HIGH_IMPACT_JOURNALS[340:355]    # 分子生物学与细胞生物学
    )

    for journal in tqdm(journals_to_search, desc="High-Impact Journals"):
        try:
            query = f"({' '.join(KEYWORDS)}) AND ({journal}[Journal])"
            url = "https://api.crossref.org/works"
            params = {
                "query": query,
                "filter": f"from-pub-date:{START_YEAR},until-pub-date:{END_YEAR}",
                "rows": 10  # 每个期刊搜索10篇
            }

            response = requests.get(url, params=params, timeout=25)
            data = response.json()

            journal_items = []
            for item in data.get("message", {}).get("items", []):
                container_title = item.get("container-title", [""])[0]
                if journal.lower() in container_title.lower():
                    abs_text = clean_html(item.get("abstract", ""))
                    journal_items.append({
                        "Title": item.get("title", [""])[0] if item.get("title") else "No title",
                        "Authors": ", ".join([a.get("family", "") for a in item.get("author", [])]) if "author" in item else "",
                        "Year": item.get("created", {}).get("date-parts", [[None]])[0][0] if item.get("created") else "",
                        "DOI": item.get("DOI", ""),
                        "Link": doi_to_link(item.get("DOI", "")),
                        "Abstract_En": abs_text,
                        "Abstract_Source": f"High-Impact: {journal}" if abs_text else "No Abstract",
                        "Source": f"High-Impact: {journal}",
                        "Journal": journal
                    })

            items.extend(journal_items)
            print(f"  📚 {journal}: found {len(journal_items)} articles")

            time.sleep(1.5)  # 增加延迟避免被封

        except Exception as e:
            print(f"⚠️ Error searching {journal}: {e}")
            continue

    print(f"✅ High-Impact Journals found {len(items)} results from {len(journals_to_search)} journals")

    # 显示期刊分类统计
    journal_stats = {}
    category_stats = {
        "顶级综合期刊": 0,
        "精神病学与神经科学": 0,
        "内分泌与代谢": 0,
        "药理学与药物发现": 0,
        "心血管科学": 0,
        "分子生物学与细胞生物学": 0
    }

    for item in items:
        journal = item["Journal"]
        journal_stats[journal] = journal_stats.get(journal, 0) + 1

        # 统计各分类的文献数量
        if journal in HIGH_IMPACT_JOURNALS[:30]:
            category_stats["顶级综合期刊"] += 1
        elif journal in HIGH_IMPACT_JOURNALS[30:130]:
            category_stats["精神病学与神经科学"] += 1
        elif journal in HIGH_IMPACT_JOURNALS[130:210]:
            category_stats["内分泌与代谢"] += 1
        elif journal in HIGH_IMPACT_JOURNALS[210:280]:
            category_stats["药理学与药物发现"] += 1
        elif journal in HIGH_IMPACT_JOURNALS[280:340]:
            category_stats["心血管科学"] += 1
        elif journal in HIGH_IMPACT_JOURNALS[340:]:
            category_stats["分子生物学与细胞生物学"] += 1

    print(f"📊 期刊分类分布:")
    for category, count in category_stats.items():
        if count > 0:
            print(f"   {category}: {count} 篇文献")

    return items

def search_springer():
    """搜索SpringerLink"""
    print("\n🔹 Searching Springer Link ...")
    try:
        query = "+AND+".join(KEYWORDS)
        url = f"https://api.springernature.com/metadata/json?q={query}&api_key=demo&p={MAX_RESULTS_PER_SOURCE//2}"

        response = requests.get(url, timeout=30)
        data = response.json()

        items = []
        for record in data.get("records", []):
            items.append({
                "Title": record.get("title", "No title"),
                "Authors": ", ".join([creator.get("creator", "") for creator in record.get("creators", [])]),
                "Year": record.get("publicationDate", "").split("-")[0] if record.get("publicationDate") else "",
                "DOI": record.get("doi", ""),
                "Link": f"https://link.springer.com/article/{record.get('doi', '')}",
                "Abstract_En": clean_html(record.get("abstract", "")),
                "Abstract_Source": "Springer" if record.get("abstract") else "No Abstract",
                "Source": "Springer"
            })

        print(f"✅ Springer found {len(items)} results")
        return items
    except Exception as e:
        print(f"⚠️ Springer search failed: {e}")
        return []

# ---------- ENHANCED SEARCH WITH KEYWORD FILTERING ----------
def search_all_sources_with_filtering():
    """搜索所有数据源并进行关键词筛选"""
    print("🚀 Starting Enhanced Literature Collection with Keyword Filtering")

    # 检查是否有已保存的数据
    progress, saved_data = check_existing_data()

    if progress and progress['stage'] == 'search_complete':
        print("✅ Using saved search results")
        all_items = saved_data['search_results']
    else:
        all_items = []
        search_functions = [
            search_crossref,
            search_pubmed,
            search_semantic,
            search_high_impact_journals,
            search_springer
        ]

        for func in search_functions:
            try:
                results = func()
                all_items.extend(results)
                print(f"📥 Added {len(results)} from {func.__name__}")

                # 每次搜索后都保存进度
                save_progress('searching', all_items)

            except Exception as e:
                print(f"⚠️ Error in {func.__name__}: {e}")

        # 搜索完成，保存最终结果
        save_progress('search_complete', all_items)

    # 关键词扩展和筛选
    expanded_keywords = smart_keyword_expansion(KEYWORDS)
    filtered_items = filter_by_keywords(all_items, expanded_keywords, min_relevance_score=1.5)

    # 分析关键词分布
    analyze_keyword_distribution(filtered_items, expanded_keywords)

    return filtered_items, expanded_keywords

# ---------- MODIFIED DOCUMENT GENERATION ----------
def generate_word_document(final_data, completion_stats, expanded_keywords=None, all_items_count=0):
    """生成Word文档，包含关键词信息"""
    doc = Document()

    # 添加文档标题
    doc.add_heading(f'文献检索报告: {" ".join(KEYWORDS)}', 0)
    doc.add_paragraph(f'检索时间: {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
    doc.add_paragraph(f'检索关键词: {", ".join(KEYWORDS)}')

    if expanded_keywords:
        doc.add_paragraph(f'扩展关键词: {", ".join(expanded_keywords)}')

    doc.add_paragraph(f'时间范围: {START_YEAR}-{END_YEAR}')
    doc.add_paragraph(f'总计相关文献: {len(final_data)} 篇')

    # 使用传递的参数
    if all_items_count > 0:
        removed_count = all_items_count - len(final_data)
        doc.add_paragraph(f'去重统计: 初始 {all_items_count} 篇 → 去重后 {len(final_data)} 篇 (移除 {removed_count} 篇重复)')
    else:
        doc.add_paragraph(f'去重统计: 去重后 {len(final_data)} 篇文献')

    doc.add_paragraph(f'摘要补全: 改进 {completion_stats["improved"]} 篇, 未变 {completion_stats["unchanged"]} 篇, 失败 {completion_stats["failed"]} 篇')

    # 添加相关性分数统计
    relevance_scores = [item.get("Relevance_Score", 0) for item in final_data]
    if relevance_scores and len(relevance_scores) > 0:
        avg_relevance = sum(relevance_scores) / len(relevance_scores)
        doc.add_paragraph(f'平均相关性分数: {avg_relevance:.2f}')

    doc.add_paragraph()

    # 高影响因子文献特别显示
    high_impact_items = [d for d in final_data if "High-Impact" in d.get("Source", "")]
    if high_impact_items:
        doc.add_heading('🎯 高影响因子文献', level=1)
        doc.add_paragraph(f'共找到 {len(high_impact_items)} 篇来自高影响因子期刊的文献')

        for i, r in enumerate(high_impact_items, 1):
            doc.add_heading(f"{i}. {r['Title']}", level=2)
            doc.add_paragraph(f"👥 Authors: {r['Authors']}")
            doc.add_paragraph(f"📅 Year: {r['Year']} | 📚 Journal: {r['Source'].replace('High-Impact: ', '')}")

            if r["DOI"]:
                p = doc.add_paragraph("🔗 DOI: ")
                make_hyperlink(p, f"https://doi.org/{r['DOI']}", r["DOI"])

            doc.add_paragraph("📄 Abstract (English):")
            doc.add_paragraph(r['Abstract_En'])
            doc.add_paragraph("📖 摘要（中文）:")
            doc.add_paragraph(r['Abstract_Zh'])
            doc.add_paragraph("-" * 60)

    # 其他文献按来源分组
    sources = sorted(list(set([d["Source"] for d in final_data if "High-Impact" not in d["Source"]])))
    for source in sources:
        source_items = [d for d in final_data if d["Source"] == source]
        if source_items:
            doc.add_heading(f'📖 {source} ({len(source_items)}篇)', level=1)

            for i, r in enumerate(source_items, 1):
                doc.add_heading(f"{i}. {r['Title']}", level=2)
                doc.add_paragraph(f"👥 Authors: {r['Authors']}")
                doc.add_paragraph(f"📅 Year: {r['Year']} | 🔍 Source: {r['Source']}")

                if r["DOI"]:
                    p = doc.add_paragraph("🔗 DOI: ")
                    make_hyperlink(p, f"https://doi.org/{r['DOI']}", r["DOI"])
                elif r["Link"]:
                    p = doc.add_paragraph("🔗 Link: ")
                    make_hyperlink(p, r["Link"], "Click to view original paper")

                doc.add_paragraph("📄 Abstract (English):")
                doc.add_paragraph(r['Abstract_En'])
                doc.add_paragraph("📖 摘要（中文）:")
                doc.add_paragraph(r['Abstract_Zh'])
                doc.add_paragraph("-" * 60)

    return doc

# ---------- MODIFIED MAIN FUNCTION ----------
def main():
    """主执行函数，包含关键词筛选和数据持久化"""

    # 阶段1: 搜索文献并进行关键词筛选
    print("=" * 60)
    print("📚 STAGE 1: Literature Search with Keyword Filtering")
    print("=" * 60)

    all_items, expanded_keywords = search_all_sources_with_filtering()
    print(f"✅ Search and filtering completed: {len(all_items)} relevant items found")

    # 阶段2: 去重
    print("\n" + "=" * 60)
    print("🔄 STAGE 2: Deduplication")
    print("=" * 60)

    progress, saved_data = check_existing_data()
    if progress and progress['stage'] == 'deduplication_complete':
        print("✅ Using saved deduplicated data")
        final_data = saved_data['final_data']
    else:
        print("🔹 Step 1: Basic deduplication...")
        basic_dedup_items = group_and_deduplicate(all_items)

        print("🔹 Step 2: Advanced similarity deduplication...")
        final_data = advanced_deduplication(basic_dedup_items)

        save_progress('deduplication_complete', final_data)

    print(f"✅ Deduplication completed: {len(final_data)} unique relevant items")

    # 阶段3: 增强摘要补全
    print("\n" + "=" * 60)
    print("🌐 STAGE 3: Enhanced Abstract Completion")
    print("=" * 60)

    completion_stats = enhanced_abstract_completion(final_data)


     # ---------- TRANSLATION ----------
    print("\n" + "=" * 60)
    print("🈺 STAGE 4: Translation")
    print("=" * 60)

    for idx, item in enumerate(tqdm(final_data, desc="Translation Progress")):
        if item["Abstract_En"] and item["Abstract_En"] != "Abstract not available from any source.":
            item["Abstract_Zh"] = translate_to_cn(item["Abstract_En"])
        else:
            item["Abstract_Zh"] = "未找到摘要"

    # 每翻译10个文献保存一次进度
        if idx % 10 == 0:
            save_progress('translation', final_data)

# 阶段5: 生成文档
    print("\n" + "=" * 60)
    print("📄 STAGE 5: Document Generation")
    print("=" * 60)

    output_name = f"{FILE_BASENAME}.docx"
    doc = generate_word_document(final_data, completion_stats, expanded_keywords, len(all_items))
    doc.save(output_name)

    # 保存到Google Drive（可选）
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        drive_path = f"/content/drive/MyDrive/{output_name}"
        doc.save(drive_path)
        print(f"✅ Also saved to Google Drive: {drive_path}")
    except:
        print("ℹ️ Google Drive not available")

    # 最终统计
    print(f"\n📊 FINAL STATISTICS:")
    print(f"   Unique relevant items: {len(final_data)}")
    high_impact_items = [d for d in final_data if "High-Impact" in d.get("Source", "")]
    print(f"   High-impact items: {len(high_impact_items)}")
    print(f"   Items with DOI: {len([d for d in final_data if d['DOI']])}")
    print(f"   Items with full abstract: {len([d for d in final_data if len(d['Abstract_En']) > 200])}")

    # 下载文件
    try:
        from google.colab import files
        files.download(output_name)
        print(f"📥 File '{output_name}' downloaded successfully!")
    except:
        print("💻 File saved in current directory")

    # 保存完成状态
    save_progress('complete', final_data)
    print(f"\n🎉 Literature collection completed successfully!")
    print(f"💾 All data saved in: {DATA_DIR}")

# ---------- 执行主函数 ----------
if __name__ == "__main__":
    main()